# Black Hat USA Training (Early draft)

## Lab 1: Adversarial EXEmple Impact on Manual Heuristics (YARA)

We will follow a "Top-Down" teaching methodology: We will start with higher level concepts familiar to our students in the cybersecurity domain, for instance, by introducing a specific library and demonstrating its use. Then, we delve deeper into the methods and parameters of these applications. Finally, we explore the underlying fundamentals, such as the specific PE format properties or mathematical concepts at the core of these ideas.

**NOTE: This is a raw draft that will be populated with more material (especially visual) and explanations, especially, facilitating AI/ML intuition and more gradual familiriaztion with concepts.**

Contents:
- When Adversarial EXEmples Help
- When Adversarial EXEmples Hurt


## When Adversarial EXEmples Help

In [90]:
# if not installed, install yara-python with pip
%pip install yara-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\dtrizna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [125]:
# force reimport of lab_helpers
import sys
if 'lab_helpers' in sys.modules:
    del sys.modules['lab_helpers']

from lab_helpers import *
yara = YaraWrapper()

In [39]:
e430_orig_path = "http://malware-training.us.to/e430acd25056f7616cee8ce7b399302658a75450077622555376e69fb6aa37ca.orig.7z"
e430_orig_bytez = get_encrypted_archive(e430_orig_path, password="infected")
e430_orig_bytez[0:20]

b'MZP\x00\x02\x00\x00\x00\x04\x00\x0f\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

In [9]:
import requests

xor_rule_url = "https://raw.githubusercontent.com/Neo23x0/signature-base/6b8e2a00e5aafcfcfc767f3f53ae986cf81f968a/yara/gen_susp_xor.yar"
xor_rule = requests.get(xor_rule_url).text
print(xor_rule)

/* requires YARA 3.8 or higher */
import "pe"

rule SUSP_XORed_URL_In_EXE {
   meta:
      description = "Detects an XORed URL in an executable"
      author = "Florian Roth (Nextron Systems)"
      reference = "https://twitter.com/stvemillertime/status/1237035794973560834"
      date = "2020-03-09"
      modified = "2022-09-16"
      score = 50
      id = "f83991c8-f2d9-5583-845a-d105034783ab"
   strings:
      $s1 = "http://" xor
      $s2 = "https://" xor
      $f1 = "http://" ascii
      $f2 = "https://" ascii

      $fp01 = "3Com Corporation" ascii  /* old driver */
      $fp02 = "bootloader.jar" ascii  /* DeepGit */
      $fp03 = "AVAST Software" ascii wide
      $fp04 = "smartsvn" wide ascii fullword
      $fp05 = "Avira Operations GmbH" wide fullword
      $fp06 = "Perl Dev Kit" wide fullword
      $fp07 = "Digiread" wide fullword
      $fp08 = "Avid Editor" wide fullword
      $fp09 = "Digisign" wide fullword
      $fp10 = "Microsoft Corporation" wide fullword
      $fp11 = "M

In [111]:
yara.check_sample(sample_bytes=e430_orig_bytez, rules=xor_rule)
yara.pretty_print()

[+] Match found: [SUSP_XORed_URL_In_EXE]
[!] Rule: SUSP_XORed_URL_In_EXE
    [+] Matched data: b'iuuq;..' | Offset: 0x68650
	[!] XOR key: 1
	[!] Decoded: b'http://'
    [+] Matched data: b'iuuq;..' | Offset: 0x68964
	[!] XOR key: 1
	[!] Decoded: b'http://'


<img src="img/e430_match_raw.png" width=500>

This:

<img src="img/e430_enc_url.png" width=500>

### Adversarial EXEmple with Padding Attack

In [40]:
e430_adv_path = "http://malware-training.us.to/e430acd25056f7616cee8ce7b399302658a75450077622555376e69fb6aa37ca.adv.7z"
e430_adv_bytez = get_encrypted_archive(e430_adv_path, password="infected")
e430_adv_bytez[0:20]

b'MZP\x00\x02\x00\x00\x00\x04\x00\x0f\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

In [104]:
yara.check_sample(sample_bytes=e430_adv_bytez, rules=xor_rule)
yara.pretty_print()

[-] No matches found.


<img src="img/e430_orig_vs_adv.png" width=400>



In [67]:
print(f"Size of orig sample: {len(e430_orig_bytez) / 1024 / 1024:^6.2f}MB")
print(f"Size of adv  sample: {len(e430_adv_bytez) / 1024 / 1024:^6.2f}MB")

Size of orig sample:  0.49 MB
Size of adv  sample: 17.15 MB


In [57]:
print(xor_rule.split("condition:")[1])


      uint16(0) == 0x5a4d and
      filesize < 2000KB and (
         ( $s1 and #s1 > #f1 ) or
         ( $s2 and #s2 > #f2 )
      )
      and not 1 of ($fp*)
      and not pe.number_of_signatures > 0
}




## When Adversarial EXEmples Hurt

### CryptoLocker rule

In [105]:
cryptolocker_rule_url = "https://raw.githubusercontent.com/advanced-threat-research/Yara-Rules/fc51a3fe3b450838614a5a5aa327c6bd8689cbb2/ransomware/RANSOM_Cryptolocker.yar"
cryptolocker_rules = requests.get(cryptolocker_rule_url).text
print(cryptolocker_rules)

rule CryptoLocker_set1
{

	meta:

		description = "Detection of Cryptolocker Samples"
		author = "Christiaan Beek, Christiaan_Beek@McAfee.com"
		date = "2014-04-13"
		rule_version = "v1"
	    malware_type = "ransomware"
	    malware_family = "Ransom:W32/Cryptolocker"
	    actor_type = "Cybercrime"
	    actor_group = "Unknown"
		
		
	strings:

		$string0 = "static"
		$string1 = " kscdS"
		$string2 = "Romantic"
		$string3 = "CompanyName" wide
		$string4 = "ProductVersion" wide
		$string5 = "9%9R9f9q9"
		$string6 = "IDR_VERSION1" wide
		$string7 = "  </trustInfo>"
		$string8 = "LookFor" wide
		$string9 = ":n;t;y;"
		$string10 = "        <requestedExecutionLevel level"
		$string11 = "VS_VERSION_INFO" wide
		$string12 = "2.0.1.0" wide
		$string13 = "<assembly xmlns"
		$string14 = "  <trustInfo xmlns"
		$string15 = "srtWd@@"
		$string16 = "515]5z5"
		$string17 = "C:\\lZbvnoVe.exe" wide

	condition:

		12 of ($string*)
}

rule CryptoLocker_rule2
{

	meta:

		description = "Detection of Crypto

In [106]:
orig_57ed_path = "http://malware-training.us.to/57edf6f6ca18699cd4667e5d314fd6370d48a3733b266857ed6123d093b064a7.orig.7z"
orig_57ed_bytez = get_encrypted_archive(orig_57ed_path, password="infected")
orig_57ed_bytez[0:20]

b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

In [107]:
yara.check_sample(sample_bytes=orig_57ed_bytez, rules=cryptolocker_rules)
yara.pretty_print()

[-] No matches found.


#### Trying with Adversarial EXEmple with Section Injection Attack

In [108]:
adv_57ed_path = "http://malware-training.us.to/57edf6f6ca18699cd4667e5d314fd6370d48a3733b266857ed6123d093b064a7.adv.7z"
adv_57ed_bytez = get_encrypted_archive(adv_57ed_path, password="infected")
adv_57ed_bytez[0:20]

b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

In [126]:
yara.check_sample(sample_bytes=adv_57ed_bytez, rules=cryptolocker_rules)
yara.pretty_print(print_limit=5)

[+] Match found: [CryptoLocker_rule2]
[!] Rule: CryptoLocker_rule2
    [+] Matched data: b'2\x00.\x000\x00.\x001\x00.\x007\x00' | Offset: 0x128681c
    [+] Matched data: b'    <security>' | Offset: 0xc3a0
    [+] Matched data: b'    <security>' | Offset: 0x17cd41
    [+] Matched data: b'    <security>' | Offset: 0x21aac8
    [+] Matched data: b'    <security>' | Offset: 0x35e576
    [+] Matched data: b'    <security>' | Offset: 0x397c5b
    [*] ... 9 more

    [+] Matched data: b'P\x00r\x00o\x00d\x00u\x00c\x00t\x00V\x00e\x00r\x00s\x00i\x00o\x00n\x00' | Offset: 0xc6fa
    [+] Matched data: b'P\x00r\x00o\x00d\x00u\x00c\x00t\x00V\x00e\x00r\x00s\x00i\x00o\x00n\x00' | Offset: 0x94be2
    [+] Matched data: b'P\x00r\x00o\x00d\x00u\x00c\x00t\x00V\x00e\x00r\x00s\x00i\x00o\x00n\x00' | Offset: 0x15ef8a
    [+] Matched data: b'P\x00r\x00o\x00d\x00u\x00c\x00t\x00V\x00e\x00r\x00s\x00i\x00o\x00n\x00' | Offset: 0x17c9a6
    [+] Matched data: b'P\x00r\x00o\x00d\x00u\x00c\x00t\x00V\x00e\x00r\x00s\x00i\x

<img src="img/57ed_orig_vs_adv.png" width=400>

In [122]:
print(cryptolocker_rules.split("condition:")[1].split("rule")[0].rstrip("} \n"))



		12 of ($string*)


It triggers on meta-data component of PE file: `manifest.xml` that appear many times, instead of appearing only once (like in any regular PE).

### ConfuserEx rule

In [130]:
confuser_ex_rule_url = "https://raw.githubusercontent.com/Neo23x0/signature-base/6b8e2a00e5aafcfcfc767f3f53ae986cf81f968a/yara/gen_github_net_redteam_tools_names.yar"
confuser_ex_rules = requests.get(confuser_ex_rule_url).text
rule_name = "rule SUSP_NET_NAME_ConfuserEx"
print(rule_name + confuser_ex_rules.split(rule_name)[1].split("rule")[0])

rule SUSP_NET_NAME_ConfuserEx {
    meta:
        description = "Detects ConfuserEx packed file"
        reference = "https://github.com/yck1509/ConfuserEx"
        license = "Detection Rule License 1.1 https://github.com/Neo23x0/signature-base/blob/master/LICENSE"
        author = "Arnim Rupp"
        score = 40
        date = "2021-01-22"
        modified = "2021-01-25"
        id = "f1bda14e-c9fe-5341-8962-691a66233eb0"
    strings:
        $name = "ConfuserEx" ascii wide
        $compile = "AssemblyTitle" ascii wide
    condition:
        (uint16(0) == 0x5A4D and uint32(uint32(0x3C)) == 0x00004550) and all of them
}




In [133]:
yara.check_sample(sample_bytes=orig_57ed_bytez, rules=confuser_ex_rules)
yara.pretty_print()

[-] No matches found.


In [135]:
yara.check_sample(sample_bytes=adv_57ed_bytez, rules=confuser_ex_rules)
yara.pretty_print()

[+] Match found: [SUSP_NET_NAME_ConfuserEx]
[!] Rule: SUSP_NET_NAME_ConfuserEx
    [+] Matched data: b'ConfuserEx' | Offset: 0x56ac27
    [+] Matched data: b'AssemblyTitle' | Offset: 0x3f59ef
    [+] Matched data: b'AssemblyTitle' | Offset: 0x56a90b
    [+] Matched data: b'A\x00s\x00s\x00e\x00m\x00b\x00l\x00y\x00T\x00i\x00t\x00l\x00e\x00' | Offset: 0x11b71ed
    [+] Matched data: b'A\x00s\x00s\x00e\x00m\x00b\x00l\x00y\x00T\x00i\x00t\x00l\x00e\x00' | Offset: 0x11b7252


In [138]:
adv_57ed_path_no_confuser = "http://malware-training.us.to/57edf6f6ca18699cd4667e5d314fd6370d48a3733b266857ed6123d093b064a7.adv.no_confuser_section.7z"
adv_57ed_path_no_confuser_bytez = get_encrypted_archive(adv_57ed_path_no_confuser, password="infected")
adv_57ed_path_no_confuser_bytez[0:20]

b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

In [137]:
yara.check_sample(sample_bytes=adv_57ed_path_no_confuser_bytez, rules=confuser_ex_rules)
yara.pretty_print()

[+] Match found: [SUSP_NET_NAME_ConfuserEx]
[!] Rule: SUSP_NET_NAME_ConfuserEx
    [+] Matched data: b'ConfuserEx' | Offset: 0x56ac27
    [+] Matched data: b'AssemblyTitle' | Offset: 0x3f59ef
    [+] Matched data: b'AssemblyTitle' | Offset: 0x56a90b
    [+] Matched data: b'A\x00s\x00s\x00e\x00m\x00b\x00l\x00y\x00T\x00i\x00t\x00l\x00e\x00' | Offset: 0x11b71ed
    [+] Matched data: b'A\x00s\x00s\x00e\x00m\x00b\x00l\x00y\x00T\x00i\x00t\x00l\x00e\x00' | Offset: 0x11b7252


ConfuserEx is a open-source protector for .NET applications. It is the successor of Confuser project. It's primarily designed to make reverse engineering difficult for applications written in .NET languages like C# and VB.NET. ConfuserEx does this by using a variety of techniques like symbol renaming, control flow obfuscation, and encryption of strings and resources.

https://github.com/yck1509/ConfuserEx:

<img src="img/confuser_repo.png" width=500>

https://www.microsoft.com/en-us/wdsi/threats/malware-encyclopedia-description?Name=Trojan:MSIL/Confuser&ThreatID=2147728476:

<img src="img/confuser_signature.png" width=500>

#### Evading this rule match

We can acknowledge that string is present:

<img src="img/confuser_bytes.png" width=500>

Section where it is present:

<img src="img/confuser_section.png" width=500>

It is possible to greedy nullify it:

<img src="img/confuser_section_clear.png" width=500>

Let's do more clever approach:

In [159]:
# parse pe file and get the section names
import pefile
from copy import deepcopy

adv_57ed_bytez_mutable = bytearray(deepcopy(adv_57ed_bytez))

pe = pefile.PE(data=adv_57ed_bytez)
substring = b"ConfuserEx"
for section in pe.sections:
    section_data = section.get_data()
    if substring in section_data:
        # print(section.Name)
        print(f"[+] Found {substring} in section {section.Name}")
        
        # nullify the substring
        start_index = section_data.find(substring)
        end_index = start_index + len(substring)

        file_offset = section.PointerToRawData + start_index
        print(f"\t[!] File offset 0x{file_offset:x}")

        # nullify the substring
        adv_57ed_bytez_mutable[file_offset:file_offset + len(substring)] = b'\x00' * len(substring)

        print(f"\t[!] Overwritten {len(substring)} bytes")

adv_57ed_bytez_no_confuser = bytes(adv_57ed_bytez_mutable)

[+] Found b'ConfuserEx' in section b'ooJOnATr'
	[!] File offset 0x56ac27
	[!] Overwritten 10 bytes


In [161]:
yara.check_sample(sample_bytes=adv_57ed_bytez_no_confuser, rules=confuser_ex_rules)
yara.pretty_print()

[-] No matches found.


### Resources

https://github.com/Neo23x0/YARA-Performance-Guidelines

https://yara.readthedocs.io/en/stable/writingrules.html